In [1]:
from udma import *
import numpy as np
import matplotlib.pyplot as plt

In [2]:
cb=UDMA_CLASS("140.105.17.180", 7)
# cb=UDMA_CLASS("192.168.1.10", 7)
cb.connect()
cb.log(0)

[(5,), 0]

Comblock Registers

|Register | Connection |
| ----- | ---- |
|reg0_o | Config_Res_intan_A|
|reg1_o | Config_Res_intan_B|
|reg2_o | Config_Res_intan_C|
|reg3_o | Config_Res_intan_D|
|reg4_o | Read_intan(3:0)|
|reg5_o | Sys_reset (active low)|
|reg6_o | **Combintan_w_header**/nsamples |
|reg7_o | **Combintan_w_header**/sys_en |





In [9]:
def hicce_init(test_mode=0):
    # Select comblock
    cb.select_comblock(0)
    # Reset HiCCE
    cb.write_reg(5, 0) #RESET ON
    
    #Disable Acquisiton 
    cb.write_reg(4, 0)
    cb.write_reg(7, 0)

    # FIFO Clear
    cb.select_comblock(0)
    cb.write_reg(17, 1)
    cb.write_reg(17, 0)
    cb.select_comblock(1)
    cb.write_reg(17, 1)
    cb.write_reg(17, 0)
    cb.select_comblock(0)

    # Configure INTANT in Sequential mode (Reset Counter)
    cb.write_reg(0, 0x1c02+test_mode)
    cb.write_reg(1, 0x1c02+test_mode)
    cb.write_reg(2, 0x1c02+test_mode)
    cb.write_reg(3, 0x1c02+test_mode)
    # Configure INTANT (Cyclic Read)
    cb.write_reg(0, 0x26+test_mode)
    cb.write_reg(1, 0x26+test_mode)
    cb.write_reg(2, 0x26+test_mode)
    cb.write_reg(3, 0x26+test_mode)

def enable_acquisition(NSamples=100):
    cb.select_comblock(0)
    #Set number of samples per package
    cb.write_reg(6, 32*NSamples)

    #Enable acquisition
    cb.write_reg(4, 15) ##Enabling INTAN readout (legacy)
    cb.write_reg(5, 1) ##Enabling HICCE Driver
    cb.write_reg(7, 1) #Enable FIFOs
    
    

def read_channels(Ncomblock=0, NSamp=100, TO=1000):
    cb.select_comblock(Ncomblock)
    wordsInFifo=cb.read_reg(34)[1][0]>>16
    #Wait until FIFO have enough samples to read.
    wd=0
    while wordsInFifo<(32*NSamp+4):
        wordsInFifo_new=cb.read_reg(34)[1][0]>>16
        if wordsInFifo_new == wordsInFifo:
            wd+=1
            if wd==TO: #if the value doesn't change after TO iterations return error
                return -1
        else:
            wordsInFifo=wordsInFifo_new
            print("Values in FIFO ", wordsInFifo)
            wd=0
    if NSamp==1:
        samples_to_read=(32*NSamp)+5
    else:
        samples_to_read=(32*NSamp)+4

    dpack=cb.read_hicce(samples_to_read)
    if dpack[0][0] == -1:
        return -1
    elif dpack[0][1] < samples_to_read:
        return -1
    else:
        return dpack[1]

def decode(dpack):
    head=dpack[0]
    CEN=(head>>16) & 0xff
    SBT=(head>>8) & 0xff
    SAT=head & 0xff
    TS=dpack[1]<<32 | dpack[2]
    TAIL=dpack[-1]
    CENT=(TAIL>>16) & 0xff
    FLAGS=TAIL & 0xffff
    
    CH=[]
    for i in range(64):
        CH.append([])
    
    for i in range(len(dpack[3:-1])):
        d=dpack[i]
        #Splitting into Most significant channel and less significant channel
        MSCH=d>>16
        LSCH=d&0xffff

        if MSCH >= 0x8000:  # Check if number is negative
            MSCH = -((MSCH ^ 0xffff) + 1) # Convert two's complement to negative number
        if LSCH >= 0x8000:  # Check if number is negative
            LSCH = -((LSCH ^ 0xffff) + 1) # Convert two's complement to negative number
        
        CH[(i%32)].append(MSCH)
        CH[(i%32)+32].append(LSCH)
         
    return CH, TS, (SBT==SAT, FLAGS==0)

In [ ]:
# cb.s.recv(1000)

In [ ]:
sampling_time=4e-9
NofIterations=50
chunksize=20

hicce_init(test_mode=0)
enable_acquisition(chunksize)
ch=[]
t=[]
for i in range(128):
    ch.append([])
for i in range(NofIterations):
    ab_raw=read_channels(0,chunksize)
    cd_raw=read_channels(1,chunksize)
    if ab_raw == -1:
        print('Error', i)
        pass
    else:
        ab_decode=decode(ab_raw)
        CHAB, TSAB, FLAGSAB=ab_decode
    if cd_raw == -1:
        pass
    else:
        CHCD, TSCD, FLAGSCD=decode(cd_raw)
    ttemp=[TSAB*sampling_time + i * sampling_time for i in range(chunksize)]
    for i in range(64):
        if FLAGSAB[0] or FLAGSAB[1] or FLAGSCD[0] or FLAGSCD[1]:
            ch[i]=ch[i]+CHAB[i]
            ch[i%64+64]=ch[i%64+64]+CHCD[i]
    t=ttemp+t

In [ ]:
def readall(NofIterations=50, chunksize=20):
    ch=[]
    tAB=[]
    tCD=[]

    samples=[]

    for i in range(128):
        ch.append([])
    for i in range(NofIterations):
        ab_raw=read_channels(0,chunksize)
        cd_raw=read_channels(1,chunksize)
        if ab_raw == -1:
            print('Error', i)
            pass
        else:
            ab_decode=decode(ab_raw)
            CHAB, TSAB, FLAGSAB=ab_decode
        if cd_raw == -1:
            pass
        else:
            CHCD, TSCD, FLAGSCD=decode(cd_raw)
        if FLAGSAB[0] or FLAGSAB[1] or FLAGSCD[0] or FLAGSCD[1]:
            samples.append(CHAB+CHCD)
            tAB.append(TSAB)
            tCD.append(TSCD)

    return samples, tAB, tCD

In [6]:
def read_HICCE(comblock=0, NofIterations=50, chunksize=20):
    ch=[]
    t=[]
    samples=[]
    for i in range(128):
        ch.append([])
    for i in range(NofIterations):
        data_raw=read_channels(comblock,chunksize)
        print(data_raw)
        if data_raw == -1:
            print('Error', i)
            pass
        else:
            CH, TS, FLAGS=decode(data_raw)
            if FLAGS[0] or FLAGS[1] :
                samples.append(CH)
                t.append(TS)
            else:
                print('Flags Error', i)

    return samples, t

In [7]:
cb.s.recv(1000)

KeyboardInterrupt: 

In [8]:
sampling_time=4e-9
NofIterations=500
chunksize=20

hicce_init(test_mode=0)
enable_acquisition(chunksize)
samples, tab, tcd=read_HICCE(0,NofIterations, chunksize)
for i in range(len(samples)):
    sample=samples[i]                
    timestamp=tab[i]*4e-9
    print(len(sample), timestamp)

KeyboardInterrupt: 

In [ ]:
# sampling_time=4e-9
# NofIterations=500
# chunksize=20

# hicce_init(test_mode=0)
# enable_acquisition(chunksize)
# samples, tab, tcd=readall(NofIterations, chunksize)
# for i in range(len(samples)):
#     sample=samples[i]                
#     timestamp=tab[i]*4e-9
#     print(len(sample), timestamp)


In [ ]:
sampling_time=4e-9
ch=[]
t=[]
for i in range(128):
    ch.append([])
for i in range(len(samples)):
    sample=samples[i]
    t0=tab[i]*sampling_time
    for j in range(128):
        ch[j]=ch[j]+sample[j]
    for i in range(len(sample[0])):
        t.append(t0+sampling_time*i)

In [ ]:
tab